In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import re
train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")
train1 = train.drop(['Descript', 'Resolution'], axis = 1)

In [14]:
train1 = train1.query('Y != 90')

In [15]:
# Case8 DayOfWeekのみone-hot train1.ver

Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(['Dates'], axis = 1, inplace = True)
def to_season(month_num):
    season = 'winter'
    if 3<=month_num<=5:
        season = 'spring'
    elif 6<=month_num<=8:
        season = 'summer'
    elif 9<=month_num<=11:
        season = 'autumn'
    return season
train1['Season'] =Dt.dt.month.apply(to_season)

count_encoding = train1['PdDistrict'].value_counts().to_dict()
train1['PdDistrict'] = train1['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)

le = LabelEncoder()
train1['Category'] = le.fit_transform(train1['Category'])

train1_DOW = pd.get_dummies(train1['DayOfWeek'])

train1 = pd.concat([train1, train1_DOW], axis = 1)

train1.drop(['DayOfWeek'], axis = 1, inplace=True)

train1.head()

,Category,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour,Season,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,37,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,spring,0,0,0,0,0,0,1
1,21,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,spring,0,0,0,0,0,0,1
2,21,105284,0,-122.424363,37.800414,2015-05-13,2015,5,13,23,spring,0,0,0,0,0,0,1
3,16,105284,1,-122.426995,37.800873,2015-05-13,2015,5,13,23,spring,0,0,0,0,0,0,1
4,16,49311,1,-122.438738,37.771541,2015-05-13,2015,5,13,23,spring,0,0,0,0,0,0,1


In [16]:
train1_SS = pd.get_dummies(train1['Season'])

train1 = pd.concat([train1, train1_SS], axis = 1)

train1.drop(['Season'], axis = 1, inplace=True)
train1.head()

,Category,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour,...,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,autumn,spring,summer,winter
0,37,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,...,0,0,0,0,0,1,0,1,0,0
1,21,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,...,0,0,0,0,0,1,0,1,0,0
2,21,105284,0,-122.424363,37.800414,2015-05-13,2015,5,13,23,...,0,0,0,0,0,1,0,1,0,0
3,16,105284,1,-122.426995,37.800873,2015-05-13,2015,5,13,23,...,0,0,0,0,0,1,0,1,0,0
4,16,49311,1,-122.438738,37.771541,2015-05-13,2015,5,13,23,...,0,0,0,0,0,1,0,1,0,0


In [17]:
# Case8 DayOfWeekのみone-hot test.ver

Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
test.drop(['Dates'], axis = 1, inplace = True)
def to_season(month_num):
    season = 'winter'
    if 3<=month_num<=5:
        season = 'spring'
    elif 6<=month_num<=8:
        season = 'summer'
    elif 9<=month_num<=11:
        season = 'autumn'
    return season
test['Season'] =Dt.dt.month.apply(to_season)


count_encoding = test['PdDistrict'].value_counts().to_dict()
test['PdDistrict'] = test['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
test['Address'] = test['Address'].replace(Ad_dict,regex = True)


test_DOW = pd.get_dummies(test['DayOfWeek'])

test = pd.concat([test, test_DOW], axis = 1)

test.drop(['DayOfWeek'], axis = 1, inplace=True)

test.head()

,Id,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour,Season,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,89591,1,-122.399588,37.735051,2015-05-10,2015,5,10,23,spring,0,0,0,1,0,0,0
1,1,89591,0,-122.391523,37.732432,2015-05-10,2015,5,10,23,spring,0,0,0,1,0,0,0
2,2,107017,1,-122.426002,37.792212,2015-05-10,2015,5,10,23,spring,0,0,0,1,0,0,0
3,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,spring,0,0,0,1,0,0,0
4,4,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,spring,0,0,0,1,0,0,0


In [18]:
test_SS = pd.get_dummies(test['Season'])

test = pd.concat([test, test_SS], axis = 1)

test.drop(['Season'], axis = 1, inplace=True)
test.head()

,Id,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour,...,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,autumn,spring,summer,winter
0,0,89591,1,-122.399588,37.735051,2015-05-10,2015,5,10,23,...,0,0,1,0,0,0,0,1,0,0
1,1,89591,0,-122.391523,37.732432,2015-05-10,2015,5,10,23,...,0,0,1,0,0,0,0,1,0,0
2,2,107017,1,-122.426002,37.792212,2015-05-10,2015,5,10,23,...,0,0,1,0,0,0,0,1,0,0
3,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,...,0,0,1,0,0,0,0,1,0,0
4,4,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,...,0,0,1,0,0,0,0,1,0,0


In [20]:
cols = ['PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour',"spring","summer","autumn","winter", 'Sunday', 'Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
X = train1[cols]
Y = train1['Category']
X_test = test[cols]

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [21]:
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

In [22]:
accuracy_score(y_val, y_pred)

0.29065472009719245

In [23]:
xgb_submission = model.predict(X_test)

In [25]:
import csv
pred = model.predict_proba(test[['PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour',"spring","summer","autumn","winter", 'Sunday', 'Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)
submission.head()

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.004367,0.145538,0.000043,0.000401,0.022369,0.001011,0.002467,0.013553,0.002747,...,0.000389,0.008435,0.000131,0.029259,5.715845e-07,0.005366,0.070833,0.184550,0.029785,0.033300
1,1,0.002308,0.049593,0.000019,0.000660,0.001525,0.000836,0.007909,0.039980,0.003710,...,0.000569,0.005997,0.000622,0.019836,3.958765e-06,0.000242,0.041746,0.088212,0.101417,0.028781
2,2,0.004209,0.065185,0.000025,0.000030,0.138225,0.000822,0.000804,0.011774,0.003551,...,0.000010,0.005081,0.000077,0.020115,1.531765e-07,0.017941,0.063181,0.140978,0.011543,0.003152
3,3,0.001848,0.148955,0.000023,0.001721,0.023257,0.002015,0.002103,0.023256,0.004129,...,0.000314,0.008408,0.000062,0.035966,3.609353e-07,0.007167,0.059527,0.152599,0.029005,0.028435
4,4,0.001848,0.148955,0.000023,0.001721,0.023257,0.002015,0.002103,0.023256,0.004129,...,0.000314,0.008408,0.000062,0.035966,3.609353e-07,0.007167,0.059527,0.152599,0.029005,0.028435


In [27]:
submission.to_csv('XGB_8.csv',index = False)